build_dataset 해당

In [ ]:
# from agents.macro_classes.macro_class_dataset import MacroAData
# 
# macro_agent = MacroAData()
# macro_agent.fetch_data()
# macro_agent.add_features()
# macro_agent.save_csv()
# macro_agent.make_close_price()

$$$$   X = macro_sercher(agent, ticker) 역할

In [ ]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


In [ ]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

In [ ]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

$$$$   macro_predictor(X) 역할

In [ ]:
pred_prices, target = macro_sub.m_predictor(X_seq)

In [ ]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

llm 생성 용.. macro_reviewer_draft

In [ ]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

In [ ]:
print(f"total_json: {total_json}")
print(f"opinion: {opinion}")

reviewer_rebut

In [ ]:
# reviewer_rebut()

[토론 테스트]

In [ ]:
from debate_ver4.agents_tmp.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

In [ ]:
debate_agent.run_dataset()

In [ ]:
# from agents.macro_classes.macro_funcs import macro_dataset
# # debate_agent.run_dataset() 에서 해당 코드가 실행안되는 경우
# #8분 정도 걸림
# macro_dataset('NVDA')

In [ ]:
for agent_id, agent in debate_agent.agents.items():
    scaler = getattr(agent, "scaler", None)
    y_scaler = getattr(scaler, "y_scaler", None)
    print(f"{agent_id}:", type(y_scaler), y_scaler)
    print(agent.ticker)
    # if agent_id =='MacroSentiAgent':
    #     print(agent.macro_df.columns[:20])


In [ ]:
opinions = debate_agent.get_opinion(0, "NVDA")

In [ ]:
print(f"opinions:{opinions}")

리뷰탈 해보기..

In [ ]:
debate_agent.get_rebuttal(1)

get_revise 해보기

In [ ]:
debate_agent.get_revise(1)

'''
에러 나오는 중..
[TechnicalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
[SentimentalAgent] fine-tuning 실패: 'NoneType' object has no attribute 'parameters'
'''

결과 도출해보기

In [ ]:
debate_agent.get_ensemble()

통합해서 한번에 실행하기: 기본 라운드는 총 3회. 클래스 정의떄 수정 가능

In [ ]:
#debate_agent.run()